In [2]:
from cdg_client import CDGClient
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd
import numpy as np
import requests

In [3]:
import time

In [4]:
API_KEY_PATH = '../../../api_key.txt'       # put your api_key path here
                                            # it shouldn't go to github
LIMIT = 250 
START_DATE = '2017-01-20T00:00:00Z'
END_DATE = '2025-01-20T00:00:00Z'
parse_xml = lambda data: ET.fromstring(data)

# get initial 250 bills
with open(API_KEY_PATH) as file:
    api_key = file.readline()
client = CDGClient(api_key, response_format='xml')

data, _ = client.get(f'bill?format=xml&limit={LIMIT}&fromDateTime={START_DATE}' \
                     f'&toDateTime={END_DATE}')
bills_text_list = xmltodict.parse(data)['api-root']['bills']['bill']
bills_text_df = pd.DataFrame(bills_text_list) # store bills into a data frame

In [5]:
bills_text_df['id'] = bills_text_df.congress+'.'+bills_text_df.type+'.'+bills_text_df.number

In [6]:
bills_text_df['full_text'] = ''
bills_text_df

,congress,type,originChamber,originChamberCode,number,url,title,updateDateIncludingText,latestAction,updateDate,id,full_text
0,119,HRES,House,H,27,https://api.congress.gov/v3/bill/119/hres/27?f...,Expressing opposition to Central Business Dist...,2025-01-13,"{'actionDate': '2025-01-09', 'text': 'Referred...",2025-01-13,119.HRES.27,
1,119,SRES,Senate,S,5,https://api.congress.gov/v3/bill/119/sres/5?fo...,A resolution notifying the House of Representa...,2025-01-15,"{'actionDate': '2025-01-06', 'text': 'Message ...",2025-01-15,119.SRES.5,
2,118,HR,House,H,9124,https://api.congress.gov/v3/bill/118/hr/9124?f...,To name the Department of Veterans Affairs com...,2025-01-14,"{'actionDate': '2025-01-04', 'text': 'Became P...",2025-01-14,118.HR.9124,
3,118,HR,House,H,8667,https://api.congress.gov/v3/bill/118/hr/8667?f...,To rename the community-based outpatient clini...,2025-01-14,"{'actionDate': '2025-01-04', 'text': 'Became P...",2025-01-14,118.HR.8667,
4,119,HRES,House,H,4,https://api.congress.gov/v3/bill/119/hres/4?fo...,Authorizing the Clerk to inform the President ...,2025-01-15,"{'actionDate': '2025-01-03', 'text': 'Motion t...",2025-01-15,119.HRES.4,
...,...,...,...,...,...,...,...,...,...,...,...,...
245,118,HR,House,H,4260,https://api.congress.gov/v3/bill/118/hr/4260?f...,Public Servants Protection and Fairness Act of...,2024-12-20,"{'actionDate': '2024-12-17', 'text': 'Referred...",2024-12-20,118.HR.4260,
246,118,HR,House,H,8957,https://api.congress.gov/v3/bill/118/hr/8957?f...,PROVE IT Act of 2024,2024-12-19,"{'actionDate': '2024-12-17', 'text': 'Referred...",2024-12-19,118.HR.8957,
247,118,HR,House,H,4261,https://api.congress.gov/v3/bill/118/hr/4261?f...,ARC Act of 2023,2024-12-20,"{'actionDate': '2024-12-17', 'text': 'Referred...",2024-12-20,118.HR.4261,
248,118,HR,House,H,8999,https://api.congress.gov/v3/bill/118/hr/8999?f...,Dialysis-Related Amyloidosis Treatment Act of ...,2024-12-19,"{'actionDate': '2024-12-17', 'text': 'Referred...",2024-12-19,118.HR.8999,


In [15]:
def get_text(bills, column):
    for i in range(len(bills)):
        bill_url = bills.iloc[i]['url'][28:].split('?')[0]
        text_url = f'{bill_url}/text'
        try:
            text_data, _ = client.get(text_url)
            text_versions_dict = xmltodict.parse(text_data)
        except:
            continue
        try:
            text_versions = text_versions_dict['api-root']['textVersions']['item']
        except: # don't always have text versions
            continue
        if type(text_versions) == list:
            text_version = text_versions[-1]
        elif type(text_versions)== dict:
            text_version = text_versions
        else: # sometimes they don't have the text through the api
            continue
        try:
            text_request = requests.get(text_version['formats']['item']\
                                                                [0]['url'])
            while text_request.status_code == 429:
                print('got timed out: waiting 1 min')
                time.sleep(60)
                text_request = requests.get(text_version['formats']['item']\
                                                                [0]['url'])
            literal_text = text_request.text
        except:
            continue
        bills.loc[i, column] = literal_text
    return bills

def get_summary(bills, column):
    if column not in bills.columns:
        bills[column] =''
    for i in range(len(bills)):
        bill_url = bills.iloc[i]['url'][28:].split('?')[0]
        summary_url = f'{bill_url}/sumamries'
        try:
            summary_data, _ = client.get(summary_url)
            summary = xmltodict.parse(summary_data)
        except:
            continue
        try:
            summary_text = summary['api-root']['summaries']['summary']['cdata']['text']
        except: # don't always have summaries available
            continue
        bills.loc[i, column] = summary_text
    return bills

In [9]:
test_data, _ = client.get('bill/119/s/2722/summaries')
test_dict = xmltodict.parse(test_data)
test_dict['api-root']['summaries']['summary']['cdata']['text']

TypeError: 'NoneType' object is not subscriptable

In [48]:
requests.get('https://www.congress.gov/118/bills/s4195/BILLS-118s4195is.htm').status_code

200

In [ ]:
# with_text.to_csv(r'C:\Users\Connor McCranie\bills_text\bill_text_seed.csv', index=False)

In [ ]:
curr_bills = pd.read_csv(r'C:\Users\Connor McCranie\bills_text\bill_summaries.csv')
offset = len(curr_bills)
bill_text_data, _ = client.get(f'bill?format=xml&limit={LIMIT}&fromDateTime={START_DATE}' \
                    f'&toDateTime={END_DATE}&offset={offset}')
bills_text_list = xmltodict.parse(bill_text_data)['api-root']['bills']['bill']
bills_text_df = pd.DataFrame(bills_text_list)
n=1


while True:    
    bills_text_df = get_summary(bills_text_df, 'summary')
    combined_bills = pd.concat([curr_bills, bills_text_df], ignore_index=True)
    # update the csv's
    combined_bills.to_csv(r'C:\Users\Connor McCranie\bills_text\bill_summaries.csv', index=False)
    # go again
    curr_bills = combined_bills
    offset = len(curr_bills)
    print(f'updated {n} times: num bill texts ={offset}')
    try:
        data, _ = client.get(f'bill?format=xml&limit={LIMIT}&fromDateTime={START_DATE}'\
                        f'&toDateTime={END_DATE}&offset={offset}')
        bills_text_list = xmltodict.parse(data)['api-root']['bills']['bill']
        bills_text_df = pd.DataFrame(bills_text_list)
        n+=1
    except:
        print('encountered error: waiting 1min')
        time.sleep(60) 

updated 1 times: num bill texts =750
updated 2 times: num bill texts =1000
updated 3 times: num bill texts =1250


most don't have summaries :(

In [16]:
curr_bills = pd.read_csv(r'C:\Users\Connor McCranie\bills_text\bill_text_valid.csv')
offset = len(curr_bills)
bill_text_data, _ = client.get(f'bill?format=xml&limit={LIMIT}&fromDateTime={START_DATE}' \
                    f'&toDateTime={END_DATE}&offset={offset}')
bills_text_list = xmltodict.parse(bill_text_data)['api-root']['bills']['bill']
bills_text_df = pd.DataFrame(bills_text_list)
n=1


while True:    
    bills_text_df['full_text']=''
    bills_text_df = get_text(bills_text_df, 'full_text')
    combined_bills = pd.concat([curr_bills, bills_text_df], ignore_index=True)
    # update the csv's
    combined_bills.to_csv(r'C:\Users\Connor McCranie\bills_text\bill_text_valid.csv', index=False)
    # go again
    curr_bills = combined_bills
    offset = len(curr_bills)
    print(f'updated {n} times: num bill texts ={offset}')
    try:
        data, _ = client.get(f'bill?format=xml&limit={LIMIT}&fromDateTime={START_DATE}'\
                        f'&toDateTime={END_DATE}&offset={offset}')
        bills_text_list = xmltodict.parse(data)['api-root']['bills']['bill']
        bills_text_df = pd.DataFrame(bills_text_list)
        n+=1
    except:
        print('encountered error: waiting 1min')
        time.sleep(60) 

updated 1 times: num bill texts =32135
updated 2 times: num bill texts =32385
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
updated 3 times: num bill texts =32635
updated 4 times: num bill texts =32885
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
got timed out: waiting 1 min
updated 5 times: num bill texts =33135
updated 6 times: num bill texts =33385
updated 7 times: num bill texts =33635
updated 8 times: num bill texts =33885
updated 9 times: num bill texts =34135
got timed o

: 

: 

In [ ]:
bill_dict['textVersions']['url'][28:]

'bill/119/hres/27/text?format=xml'

In [ ]:
xmltodict.parse(data)['api-root']['bill']['textVersions']

{'count': '3',
 'url': 'https://api.congress.gov/v3/bill/119/s/2403/text?format=xml'}

In [52]:
summary_data, _ = client.get('bill/119/s/2403/?format=xml')
sum_dict = xmltodict.parse(summary_data)


In [53]:
xmltodict.parse(sum_dict)['api-root']

TypeError: a bytes-like object is required, not 'dict'

In [ ]:
url = 'https://www.congress.gov/119/bills/s2403/BILLS-119s2403es.htm'
response = requests.get(url)

In [ ]:
test_data, _ = client.get('bill/119/hres/27/text?format=xml')